# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This m

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories,
                                      data_home='~/Desktop/EugeneTang/Grad School/Berkeley/W207_AML/Assignments/Datasets')
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories,
                                     data_home='~/Desktop/EugeneTang/Grad School/Berkeley/W207_AML/Assignments/Datasets')

num_test = len(newsgroups_test.target)
dev_test_split = int(num_test/2)
test_data, test_labels = newsgroups_test.data[dev_test_split:], newsgroups_test.target[dev_test_split:]
dev_data, dev_labels = newsgroups_test.data[:dev_test_split], newsgroups_test.target[:dev_test_split]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

[2 pts]

In [20]:
id_to_category = newsgroups_train.target_names
def P1(num_examples=5):
    print('------------------------------')
    for i in range(num_examples):
        print('Label: {}'.format(id_to_category[train_labels[i]]))
        print('Text: {}'.format(train_data[i]))
        print('------------------------------')
P1(5) # change this from 2 to 5?

------------------------------
Label: comp.graphics
Text: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
------------------------------
Label: talk.religion.misc
Text: 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh 

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

[6 pts]

In [31]:
def P2():
    cv = CountVectorizer()
    train_vectors = cv.fit_transform(train_data)
    
    # a
    num_rows = train_vectors.shape[0]
    vocab_size = train_vectors.shape[1]
    print('a.')
    print('The vocabulary contains {} terms.'.format(vocab_size))
    print('The average number of non-zero features per example is {}.'.format(train_vectors.nnz/num_rows))
    print('The fraction of non-zero entries in the matrix is {}.'.format(train_vectors.nnz/(vocab_size * num_rows)))
    print()

    # b
    feature_names = cv.get_feature_names() # feature names is alphabetical already
    print('b.')
    print('The 0th feature string is {}'.format(feature_names[0]))
    print('The last feature string is {}'.format(feature_names[-1]))
    print()

    # c
    cv_c = CountVectorizer(vocabulary=["atheism", "graphics", "space", "religion"])
    train_vectors_c = cv_c.fit_transform(train_data)
    num_rows_c = train_vectors_c.shape[0]
    vocab_size_c = train_vectors_c.shape[1]
    print('c.')
    print('Our data contains {} rows and a vocabulary size of {}, as expected.'.format(num_rows_c, vocab_size_c))
    print('The fraction of non-zero entries in the matrix is {}.'.format(train_vectors_c.nnz/(vocab_size_c * num_rows_c)))
    print()
    
    # d
    cv_d = CountVectorizer(analyzer='char', ngram_range=(2,3))
    train_vectors_d = cv_d.fit_transform(train_data)
    vocab_size_d = train_vectors_d.shape[1]
    print('d.')
    print('Our character bigram/trigram data has a vocabulary size of {}.'.format(vocab_size_d))
    print()
    
    # e
    cv_e = CountVectorizer(min_df=10)
    train_vectors_e = cv_e.fit_transform(train_data)
    vocab_size_e = train_vectors_e.shape[1]
    print('e.')
    print('Our data with a minimum document frequency of 10 has a vocabulary size of {}.'.format(vocab_size_e))
    print()    
    
    # f
    cv_dev = CountVectorizer()
    dev_vectors = cv_dev.fit_transform(dev_data)
    
    dev_feature_names = cv_dev.get_feature_names()
    num_missing_words_dev = len(set(dev_feature_names) - set(feature_names))
    num_words_dev = len(dev_feature_names)
    print('f.')
    print('{} of the words in the dev data are missing from the vocabulary'.format(num_missing_words_dev/num_words_dev))
    
P2()

a.
The vocabulary contains 26879 terms.
The average number of non-zero features per example is 96.70599803343165.
The fraction of non-zero entries in the matrix is 0.0035978272269590263.

b.
The 0th feature string is 00
The last feature string is zyxel

c.
Our data contains 2034 rows and a vocabulary size of 4, as expected.
The fraction of non-zero entries in the matrix is 0.06710914454277286.

d.
Our character bigram/trigram data has a vocabulary size of 35478.

e.
Our data with a minimum document frequency of 10 has a vocabulary size of 3064.

f.
0.24787640034470024 of the words in the dev data are missing from the vocabulary


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

[4 pts]

In [68]:
def P3():
    cv = CountVectorizer()
    train_vectors = cv.fit_transform(train_data)
    dev_vectors = cv.transform(dev_data)
    
    f1_metric = 'micro' # caculate the f-score based on num-right v. num-wrong
    
    # Grid-search on KNN. We try nearest neighbors from 1 and[10,300] in intervals of 10
    n_neighbors = [1] + list(range(10,301,10))
    knn_grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid={'n_neighbors': n_neighbors}, scoring='f1_{}'.format(f1_metric))
    knn_grid.fit(train_vectors, train_labels)
    dev_pred_knn = knn_grid.best_estimator_.predict(dev_vectors)
    knn_f1_score = metrics.f1_score(dev_labels, dev_pred_knn, average=f1_metric)
    print('The optimal value for k in knn is {}.'.format(knn_grid.best_params_['n_neighbors']))
    print('The f1 score of the optimal knn classifier on the dev data is {}.'.format(knn_f1_score))
    print()
    
    # Grid-search on Multinomial Naive Bayes. We try alpha in this array: [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
    alphas = [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
    mnb_grid = GridSearchCV(estimator=MultinomialNB(), param_grid={'alpha': alphas}, scoring='f1_{}'.format(f1_metric))
    mnb_grid.fit(train_vectors, train_labels)
    dev_pred_mnb = mnb_grid.best_estimator_.predict(dev_vectors)
    mnb_f1_score = metrics.f1_score(dev_labels, dev_pred_mnb, average=f1_metric)
    print('The optimal value for alpha in Multinomial Naive Bayes is {}.'.format(mnb_grid.best_params_['alpha']))
    print('The f1 score of the optimal Mulitnomial Naive Bayes classifier on the dev data is {}.'.format(mnb_f1_score))
    
    # Grid-search on logistic regression with l2 regularization. We try the following values for C: [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
    Cs = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
    lr_grid = GridSearchCV(estimator=LogisticRegression(), param_grid={'C': Cs}, scoring='f1_{}'.format(f1_metric))
    lr_grid.fit(train_vectors, train_labels)
    dev_pred_lr = lr_grid.best_estimator_.predict(dev_vectors)
    lr_f1_score = metrics.f1_score(dev_labels, dev_pred_lr, average=f1_metric)
    print('The optimal value for C in logistic regression is {}.'.format(lr_grid.best_params_['C']))
    print('The f1 score of the optimal logistic regression classifier on the dev data is {}.'.format(lr_f1_score))
    print()
    
    print('Sum of Squared Weights by Value of C')
    print('C       | Sum of Squared Weights')
    print('--------------------------------')
    for c in Cs:
        lr = LogisticRegression(C=c)
        lr.fit(train_vectors, train_labels)
        print('{:7.4f} | {:22.2f}'.format(c, float(np.sum(np.square(lr.coef_)))))
    
P3()

The optimal value for k in knn is 150.
The f1 score of the optimal knn classifier on the dev data is 0.4423076923076923.

The optimal value for alpha in Multinomial Naive Bayes is 0.01.
The f1 score of the optimal Mulitnomial Naive Bayes classifier on the dev data is 0.7795857988165681.


/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/Users/eugenetang/miniconda3/envs/W207/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


The optimal value for C in logistic regression is 0.5.
The f1 score of the optimal logistic regression classifier on the dev data is 0.7159763313609467.

Sum of Squared Weights by Value of C
C       | Sum of Squared Weights
--------------------------------
 0.0001 |                   0.04
 0.0010 |                   0.73
 0.0100 |                  10.59
 0.1000 |                 102.27
 0.5000 |                 373.73
 1.0000 |                 601.66
 2.0000 |                 922.25
10.0000 |                2107.36


ANSWER:


a. Nearest neighbors doesn't work well for this problem because different texts in the same category will still use different words, even if those words are close in meaning. Thus a high "distance" can be misleading in this space. (e.g. if one document says "religion is bad", and the other says "I like Christianity", they will be seen as having no overlap even though they talk about very similar things.). Also, the default KNN uses Euclidean distance, so the length of the document could also influence the perceived "distances" (longer documents will be more likely to be "far" away from each other).


b. Because Naive Bayes tries to estimate probabilities that a word appears in a document of a given category, it likely requires less data to provide a decent estimate. On the other hand, in this case logistic regression may do worse because it does not have examples of each word associated with each category to make a good coefficient.


c. From the output above, we can see that the sum of the squared weights generally increase with C. This makes sense because a lower value of C penalizes higher weights more. (C is the inverse of regularization strength).

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

[5 pts]

In [17]:
def print_top_5_features(cv, lr):
    # Print out top 5 weighted features (not in absolute value: top features that indicate that a category is present)  
    top_feature_indices = []

    # Get top features
    for i in range(len(newsgroups_train.target_names)):
        top_feature_indices += lr.coef_[i].argsort()[-5:].tolist()
    
    # Print table
    print('Word         | Weight for Atheism | Weight for Computer Graphics | Weight for Space | Weight for Religion')
    print('---------------------------------------------------------------------------------------------------------')
    feature_names = cv.get_feature_names()
    for feature_index in top_feature_indices:
        # get feature name
        table_entries = [feature_names[feature_index]] 
        # add feature weights for each category
        table_entries += [lr.coef_[i][feature_index] for i in range(len(newsgroups_train.target_names))]
        print('{:12s} | {:18.6f} | {:28.6f} | {:16.6f} | {:19.6f}'.format(*table_entries))

def P4():
    # We train a logistic regression with C=0.5 (found before)
    cv = CountVectorizer()
    train_vectors = cv.fit_transform(train_data)
    lr = LogisticRegression(C=0.5)
    lr.fit(train_vectors, train_labels)
    print('Top features for each category with a unigram tokenizer')
    print_top_5_features(cv, lr)
    
    print()
        
    # Same as above except using bigram features
    cv_bigram = CountVectorizer(ngram_range=(2,2))
    train_vectors_bigram = cv_bigram.fit_transform(train_data)
    lr_bigram = LogisticRegression(C=0.5)
    lr_bigram.fit(train_vectors_bigram, train_labels)
    print('Top features for each category with a bigram tokenizer')
    print_top_5_features(cv_bigram, lr_bigram)
    
    
P4()

Top features for each category with a unigram tokenizer
Word         | Weight for Atheism | Weight for Computer Graphics | Weight for Space | Weight for Religion
---------------------------------------------------------------------------------------------------------
deletion     |           0.723524 |                    -0.249531 |        -0.272433 |           -0.270263
atheism      |           0.794488 |                    -0.348444 |        -0.367204 |           -0.377076
religion     |           0.798243 |                    -0.508671 |        -0.658825 |           -0.050695
bobby        |           0.823571 |                    -0.191204 |        -0.286357 |           -0.387236
atheists     |           0.833600 |                    -0.093277 |        -0.267655 |           -0.637544
computer     |           0.071498 |                     0.843819 |        -0.569566 |           -0.396290
3d           |          -0.303361 |                     0.941459 |        -0.571030 |           

ANSWER: 


For the unigram classifier, we see that the top features for each category are words that we would expect to associate with the category (e.g. "nasa" and "orbit" with Space, and "image" and "graphics" with Computer Graphics). Some of the top unigram features are a little bit surprising (e.g. "fbi" for Religion and "deletion" for Atheism), overall they make sense. The presense of stranger words could be the result of the content of the sample documents (e.g. fbi investigations into religious organizations).


For the bigrams features, however, we do have some surprising top features. For example, in Atheism "are you" and "you are" show up as top features, which is quite unexpected. TODO: continue

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

[4 pts]

In [7]:
def empty_preprocessor(s):
    return s

#def better_preprocessor(s):
### STUDENT START ###

### STUDENT END ###

#def P5():
### STUDENT START ###

    
### STUDENT END ###
#P5()

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

[4 pts]

In [8]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###

    

    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

[4 pts]

In [11]:
#def P7():
    ### STUDENT START ###



    ### STUDENT END ###
#P7()

ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

- [1 pt] for a reasonable attempt
- [2 pts] for improved performance